# Merge Scraped .csv Files

First we need to import some dependencies to help us.  
  
We can use `csv` and `glob` for the merge and `Pandas` for creating a **DataFrame**.

In [32]:
import csv
import glob
import shutil

import pandas as pd
# To show all columns
# pd.set_option('display.max_columns', None)

We can now add the file path of the **.csv** files to a **variable** using `glob`.  
  
We use **'*'** to access all .csv files in the folder.

In [33]:
filenames = glob.glob('../../data/dti-competitive-index/1-per-city/*.csv')

We can now create a new **.csv** file, add the **header** row from one of the **.csv**  
files and then **iterate** through all of the **.csv** files using `next` to ignore  
the first row that contains the header row.

In [34]:
with open("merged_city_data.csv", "w", newline="") as f:
    writer = csv.writer(f)
    # write the header row once (This is a manual process of copy and paste)
    writer.writerow(['Pillar / Indicator',2014,2015,2016,2017,2018,2019,2020,2021,2022,'City_Municipality'])
    for filename in filenames:
        with open(filename, "r") as infile:
            reader = csv.reader(infile)
            # skip the header row in each file using `next`
            next(reader)
            for row in reader:
                writer.writerow(row)

In [35]:
filenames.sort()  # glob lacks reliable ordering, so impose your own if output order matters
with open('merged_city_data2.csv', 'wb') as f:
    for i, fname in enumerate(filenames):
        with open(fname, 'rb') as g:
            if i != 0:
                g.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
            shutil.copyfileobj(g, f)

## Create a DataFrame

We can now check our data.

We will need to drop the column **Unnamed: 0**. This column is created because there is a missing header name in the .csv files.  
  
We can use a simple `del` to do the job.

In [37]:
del df['Unnamed: 0']
df.head(2)

,Pillar / Indicator,2014,2015,2016,2017,2018,2019,2020,2021,2022,City_Municipality
0,Local Economy Size,0.2443,0.1057,0.2782,0.0039,0.0072,0.0051,0.0013,0.0067,0.0105,Aborlan
1,Local Economy Growth,0.1861,0.1973,0.1708,0.0012,0.0224,0.0024,0.1419,0.0045,0.0001,Aborlan


Lets check the shape of the data.

In [38]:
df.shape

(81600, 11)

## Missing Values
### We have set a threshold of 50% missing values
Lets check for **Missing** values.

In [39]:
df.isnull().sum()

Pillar / Indicator    0
2014                  0
2015                  0
2016                  0
2017                  0
2018                  0
2019                  0
2020                  0
2021                  0
2022                  0
City_Municipality     0
dtype: int64

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81600 entries, 0 to 81599
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Pillar / Indicator  81600 non-null  object
 1   2014                81600 non-null  object
 2   2015                81600 non-null  object
 3   2016                81600 non-null  object
 4   2017                81600 non-null  object
 5   2018                81600 non-null  object
 6   2019                81600 non-null  object
 7   2020                81600 non-null  object
 8   2021                81600 non-null  object
 9   2022                81600 non-null  object
 10  City_Municipality   81600 non-null  object
dtypes: object(11)
memory usage: 6.8+ MB


These results are misleading as we know by a quick glance at the Dataframe shows missing values.  
  
If we look closely, the missing values contain the **"-"** character.  
  
Lets investigate using **"-"** as our search query.

In [41]:
for column in df.columns:
    count = df[column].str.count("-").sum()
    perc = round(df[column].str.count("-").sum() / 81600 * 100, 2)
    print(f"Column {column}: {count} characters, {perc}%")

Column Pillar / Indicator: 1632 characters, 2.0%
Column 2014: 68300 characters, 83.7%
Column 2015: 52636 characters, 64.5%
Column 2016: 42882 characters, 52.55%
Column 2017: 22200 characters, 27.21%
Column 2018: 21360 characters, 26.18%
Column 2019: 21040 characters, 25.78%
Column 2020: 21080 characters, 25.83%
Column 2021: 17400 characters, 21.32%
Column 2022: 161 characters, 0.2%
Column City_Municipality: 1250 characters, 1.53%


We have three columns with 50% or more missing values.  
  
Lets drop these columns.

In [42]:
df.drop(columns=['2014', '2015', '2016',], inplace=True)

And check again to confirm we have dropped the columns.

In [43]:
for column in df.columns:
    count = df[column].str.count("-").sum()
    perc = round(df[column].str.count("-").sum() / 81600 * 100, 2)
    print(f"Column {column}: {count} characters, {perc}%")

Column Pillar / Indicator: 1632 characters, 2.0%
Column 2017: 22200 characters, 27.21%
Column 2018: 21360 characters, 26.18%
Column 2019: 21040 characters, 25.78%
Column 2020: 21080 characters, 25.83%
Column 2021: 17400 characters, 21.32%
Column 2022: 161 characters, 0.2%
Column City_Municipality: 1250 characters, 1.53%


In [44]:
df.shape

(81600, 8)

Now we can work on the dataframe to convert the years into a single column and map the values.  
  
First we will create a list of column names.

In [45]:
cols = df.columns.values.tolist()
cols

['Pillar / Indicator',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 'City_Municipality']

By setting **'City_Municipality'** and **'Pillar / Indicator'** as indexes we can stack the years and values.  
  
We will create a new DataFrame **df1** in the process.

In [46]:
df1 = df[cols].set_index(['City_Municipality', 'Pillar / Indicator']).stack().reset_index()
df1

,City_Municipality,Pillar / Indicator,level_2,0
0,Aborlan,Local Economy Size,2017,0.0039
1,Aborlan,Local Economy Size,2018,0.0072
2,Aborlan,Local Economy Size,2019,0.0051
3,Aborlan,Local Economy Size,2020,0.0013
4,Aborlan,Local Economy Size,2021,0.0067
...,...,...,...,...
489595,Zumarraga,New Technology,2018,-
489596,Zumarraga,New Technology,2019,-
489597,Zumarraga,New Technology,2020,-
489598,Zumarraga,New Technology,2021,-


Lets rename the columns.

In [47]:
df1 = df1.rename(columns={'level_2': 'Year', 0: 'Value'})
df1.head(1)

,City_Municipality,Pillar / Indicator,Year,Value
0,Aborlan,Local Economy Size,2017,0.0039


We are now ready to convert the **Pillar / Indicator** values into column names (features).

In [48]:
df2 = df1.set_index(['City_Municipality', 'Year', 'Pillar / Indicator'], drop=True).unstack('Pillar / Indicator')
df2

Value  \
Pillar / Indicator     Accommodation Capacity   
City_Municipality Year                          
Aborlan           2017                 0.0000   
                  2018                 0.0000   
                  2019                 0.0000   
                  2020                 0.0000   
                  2021                 0.0000   
...                                       ...   
Zumarraga         2018                 0.0000   
                  2019                 0.0104   
                  2020                 0.0000   
                  2021                 0.0000   
                  2022                    0.0   

                                                              \
Pillar / Indicator     Active Establishments in the Locality   
City_Municipality Year                                         
Aborlan           2017                                0.2120   
                  2018                                0.2337   
                  2019                                0.0000   
                  2020                                0.1653   
                  2021                                0.2606   
...                                                      ...   
Zumarraga         2018                                0.0004   
                  2019                                0.0093   
                  2020                                1.0019   
                  2021                                0.1886   
                  2022                                0.2099   

                                              \
Pillar / Indicator     Annual Disaster Drill   
City_Municipality Year                         
Aborlan           2017                2.5000   
                  2018                0.0000   
                  2019                2.2500   
                  2020                2.5000   
                  2021                1.2542   
...                                      ...   
Zumarraga         2018                0.0000   
                  2019                2.4107   
                  2020                2.5000   
                  2021                0.0000   
                  2022                   0.0   

                                                               \
Pillar / Indicator     Availability of Basic Internet Service   
City_Municipality Year                                          
Aborlan           2017                                      -   
                  2018                                      -   
                  2019                                      -   
                  2020                                      -   
                  2021                                      -   
...                                                       ...   
Zumarraga         2018                                      -   
                  2019                                      -   
                  2020                                      -   
                  2021                                      -   
                  2022                                    0.0   

                                                                         \
Pillar / Indicator     Availability of Basic Utilities Budget for DRRMP   
City_Municipality Year                                                    
Aborlan           2017                          2.5000           0.0017   
                  2018                          0.0000           0.0000   
                  2019                          2.1047           0.2189   
                  2020                          2.3125           0.0852   
                  2021                          2.2948           0.0215   
...                                                ...              ...   
Zumarraga         2018                          0.0000           0.0000   
                  2019                          1.4719           0.0632   
                  2020                         

We have created a **Multindex** DataFrame, we need to convert back to a single index DataFrame.  
  
First we need a list of the new column names we have created. 

In [49]:
c_names = df['Pillar / Indicator'].unique().tolist()
c_names

['Local Economy Size',
 'Local Economy Growth',
 'Active Establishments in the Locality',
 'Safety Compliant Business',
 'Employment Generation',
 'Cost of Living',
 'Cost of Doing Business',
 'Financial Deepening',
 'Productivity',
 'Presence of Business and Professional Organizations',
 'Compliance to National Directives',
 'Presence of Investment Promotion Unit',
 'Compliance to ARTA Citizens Charter',
 'Capacity to Generate Local Resource',
 'Capacity of Health Services',
 'Capacity of School Services',
 'Recognition of Performance',
 'Getting Business Permits',
 'Peace and Order',
 'Social Protection',
 'Road Network',
 'Distance to Ports',
 'Availability of Basic Utilities',
 'Transportation Vehicles',
 'Education',
 'Health',
 'LGU Investment',
 'Accommodation Capacity',
 'Information Technology Capacity',
 'Financial Technology Capacity',
 'Land Use Plan',
 'Disaster Risk Reduction Plan',
 'Annual Disaster Drill',
 'Early Warning System',
 'Budget for DRRMP',
 'Local Risk Asses

We can now use this list to map and join the indexes.

In [50]:
df2.reset_index( drop=False, inplace=True)
df2.reindex([
    'Local Economy Size',
	'Local Economy Growth',
	'Active Establishments in the Locality',
	'Safety Compliant Business',
	'Employment Generation',
	'Cost of Living',
	'Cost of Doing Business',
	'Financial Deepening',
	'Productivity',
	'Presence of Business and Professional Organizations',
	'Compliance to National Directives',
	'Presence of Investment Promotion Unit',
	'Compliance to ARTA Citizens Charter',
	'Capacity to Generate Local Resource',
	'Capacity of Health Services',
	'Capacity of School Services',
	'Recognition of Performance',
	'Getting Business Permits',
	'Peace and Order',
	'Social Protection',
	'Road Network',
	'Distance to Ports',
	'Availability of Basic Utilities',
	'Transportation Vehicles',
	'Education',
	'Health',
	'LGU Investment',
	'Accommodation Capacity',
	'Information Technology Capacity',
	'Financial Technology Capacity',
	'Land Use Plan',
	'Disaster Risk Reduction Plan',
	'Annual Disaster Drill',
	'Early Warning System',
	'Budget for DRRMP',
	'Local Risk Assessments',
	'Emergency Infrastructure',
	'Utilities',
	'Employed Population',
	'Sanitary System',
	'Start Up and Innovation Facilities',
	'Innovation Financing: R&D Expenditures Allotment',
	'STEM graduates',
	'Intellectual Property Registration',
	'ICT Use: E-BPLS Software',
	'Internet Capability',
	'Availability of Basic Internet Service',
	'Online Payment Facilities',
	'ICT Plan',
	'New Technology'], axis=1)
df2

City_Municipality  Year                  Value  \
Pillar / Indicator                         Accommodation Capacity   
0                            Aborlan  2017                 0.0000   
1                            Aborlan  2018                 0.0000   
2                            Aborlan  2019                 0.0000   
3                            Aborlan  2020                 0.0000   
4                            Aborlan  2021                 0.0000   
...                              ...   ...                    ...   
9787                       Zumarraga  2018                 0.0000   
9788                       Zumarraga  2019                 0.0104   
9789                       Zumarraga  2020                 0.0000   
9790                       Zumarraga  2021                 0.0000   
9791                       Zumarraga  2022                    0.0   

                                                          \
Pillar / Indicator Active Establishments in the Locality   
0                                                 0.2120   
1                                                 0.2337   
2                                                 0.0000   
3                                                 0.1653   
4                                                 0.2606   
...                                                  ...   
9787                                              0.0004   
9788                                              0.0093   
9789                                              1.0019   
9790                                              0.1886   
9791                                              0.2099   

                                          \
Pillar / Indicator Annual Disaster Drill   
0                                 2.5000   
1                                 0.0000   
2                                 2.2500   
3                                 2.5000   
4                                 1.2542   
...                                  ...   
9787                              0.0000   
9788                              2.4107   
9789                              2.5000   
9790                              0.0000   
9791                                 0.0   

                                                           \
Pillar / Indicator Availability of Basic Internet Service   
0                                                       -   
1                                                       -   
2                                                       -   
3                                                       -   
4                                                       -   
...                                                   ...   
9787                                                    -   
9788                                                    -   
9789                                                    -   
9790                                                    -   
9791                                                  0.0   

                                                                     \
Pillar / Indicator Availability of Basic Utilities Budget for DRRMP   
0                                           2.5000           0.0017   
1                                           0.0000           0.0000   
2                                           2.1047           0.2189   
3                                           2.3125           0.0852   
4                                           2.2948           0.0215   
...                                            ...              ...   
9787                                        0.0000           0.0000   
9788                                        1.4719           0.0632   
9789                                        1.9531           0.0636   
9790                                        1.3688           0.0039   
9791                                        0.2666           0.0017   

                                                 

Time to `map` and `join` the DataFrame. We will remove all the words **'Value'** from the column names and remove the whitespace from the start and end of the names.

In [51]:
df2.columns = df2.columns.map(' '.join)
df2.columns = df2.columns.str.replace(r'Value', '')
df2.columns = df2.columns.str.lstrip()
df2.columns = df2.columns.str.rstrip()
df2.columns = df2.columns.str.lower()
df2.columns = df2.columns.str.replace(' ', '_')

df2

,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_internet_service,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,...,productivity,recognition_of_performance,road_network,stem_graduates,safety_compliant_business,sanitary_system,social_protection,start_up_and_innovation_facilities,transportation_vehicles,utilities
0,Aborlan,2017,0.0000,0.2120,2.5000,-,2.5000,0.0017,0.1098,0.2879,...,0.0145,0.2879,0.0000,-,0.1943,1.5706,0.1104,-,0.0204,0.9949
1,Aborlan,2018,0.0000,0.2337,0.0000,-,0.0000,0.0000,0.0000,0.0000,...,0.0047,0.0000,0.0000,-,0.1253,0.0000,0.0000,-,0.0000,0.0000
2,Aborlan,2019,0.0000,0.0000,2.2500,-,2.1047,0.2189,0.0614,0.1451,...,0.0004,0.0595,0.1006,-,0.2553,1.4921,0.0007,-,0.0029,1.8558
3,Aborlan,2020,0.0000,0.1653,2.5000,-,2.3125,0.0852,0.0587,0.0488,...,0.0413,0.0023,0.0006,-,0.1643,0.9946,0.0024,-,0.0026,1.8706
4,Aborlan,2021,0.0000,0.2606,1.2542,-,2.2948,0.0215,0.1064,0.0145,...,0.0690,0.0000,0.1846,-,0.1683,1.2502,0.1047,-,0.0030,1.2478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9787,Zumarraga,2018,0.0000,0.0004,0.0000,-,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,-,0.0000,0.0000,0.0000,-,0.0000,0.0000
9788,Zumarraga,2019,0.0104,0.0093,2.4107,-,1.4719,0.0632,0.0324,0.2339,...,0.0033,0.0000,0.0202,-,0.0000,1.0144,0.7160,-,0.0014,1.8799
9789,Zumarraga,2020,0.0000,1.0019,2.5000,-,1.9531,0.0636,0.1629,0.1926,...,0.0298,0.2262,0.0024,-,0.0000,1.5045,0.1281,-,0.0015,1.8754
9790,Zumarraga,2021,0.0000,0.1886,0.0000,-,1.3688,0.0039,0.0830,0.1112,...,0.0000,0.0000,0.0126,-,0.0000,1.2567,1.6311,-,0.0034,1.0391


Delete the unwanted columns.

In [53]:
del df2[
	'start_up_and_innovation_facilities'],df2[
		'new_technology'],df2[
			'online_payment_facilities'],df2[
				'ict_plan'],df2[
					'ict_use:_e-bpls_software'],df2[
						'information_technology_capacity'],df2[
							'innovation_financing:_r&d_expenditures_allotment'],df2[
								'intellectual_property_registration'],df2[
									'internet_capability'],df2[
										'stem_graduates']
df2.head(2)

,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_internet_service,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,...,presence_of_business_and_professional_organizations,presence_of_investment_promotion_unit,productivity,recognition_of_performance,road_network,safety_compliant_business,sanitary_system,social_protection,transportation_vehicles,utilities
0,Aborlan,2017,0.0000,0.2120,2.5000,-,2.5000,0.0017,0.1098,0.2879,...,0.0000,2.5000,0.0145,0.2879,0.0000,0.1943,1.5706,0.1104,0.0204,0.9949
1,Aborlan,2018,0.0000,0.2337,0.0000,-,0.0000,0.0000,0.0000,0.0000,...,0.0218,0.0000,0.0047,0.0000,0.0000,0.1253,0.0000,0.0000,0.0000,0.0000


In [28]:
df2.shape

(9792, 42)

## Export to a .csv file

In [54]:
df2.to_csv('complete_dataset.csv', index=False)

### Create a .csv file of unique City_Municipality values

In [56]:
city_unique = pd.DataFrame(df2['city_municipality'].unique()).to_csv('city_names.csv', index=False)
